In [1]:
import rioxarray
import geopandas as gpd
import pandas as pd

here's a starting point comparison https://github.com/chudasama-bijal/QGIS-Plugin-Weights-of-Evidence-Model

Get training data vector data - 1s

Classes
- Favourable
- Unfavourable

Get list of evidence rasters

Studentized t contrast

Calculate weights
